# ICESat-2 ATL15 Viewer
Interactive visualization and data extraction tool for ICESat-2 Gridded Land Ice Height Change data (ATL15)

#### Load necessary packages

In [ ]:
import os
import ipywidgets
import rioxarray
import IS2view
# autoreload
%load_ext autoreload
%autoreload 2

#### Set parameters for ATL15
- Release: ATL15 data release (001)
- Region: ATL15 data region (AA, CN, CS, GL, IS, RA, SV)
- Resolution: ATL15 horizontal resolution (01km, 10km, 20km, 40km)
- Group: ATL15 data group to read from file

In [ ]:
IS2widgets = IS2view.widgets()
ipywidgets.VBox([
    IS2widgets.release,
    IS2widgets.region,
    IS2widgets.resolution,
    IS2widgets.group,
])

### Interactive Mapping with Leaflet

Interactive maps within IS2view are built upon [ipyleaflet](https://ipyleaflet.readthedocs.io). There are 2 projections available for mapping ([North](https://epsg.io/3413) and [South](https://epsg.io/3031)) depending on the ATL15 region selected. The available basemaps are NASA's Next Generation Blue Marble visualization for the Arctic and Antarctic regions.

Regional time series can be extracted by interactively drawing geometries on the leaflet map or programmatically using [geopandas GeoDataFrames](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.html).

In [ ]:
m = IS2view.leaflet(IS2widgets.projection,
    center=IS2widgets.center,
    zoom=IS2widgets.zoom,
    draw_control=True,
    attribution=True)
m.map

### Query CMR for ATL15 granule
Eventually will switch this to the s3 instance when publicly available

In [ ]:
ids,urls = IS2view.utilities.cmr(product='ATL15',
    release=IS2widgets.release.value,
    regions=IS2widgets.region.value,
    resolutions=IS2widgets.resolution.value,
    provider='NSIDC_ECS')
if not os.access(ids[0], os.F_OK):
    IS2view.utilities.from_nsidc(urls[0], local=ids[0])

### Read and inspect ATL15 data

In [ ]:
ds = rioxarray.open_rasterio(ids[0],
    group=IS2widgets.group.value,
    masked=True)
ds

In [ ]:
IS2widgets.set_variables(ds)
IS2widgets.set_atl15_defaults()
IS2widgets.set_lags(ds)
ipywidgets.VBox([
    IS2widgets.variable,
    IS2widgets.timelag,
    IS2widgets.dynamic,
    IS2widgets.range,
    IS2widgets.cmap,
    IS2widgets.reverse,
])

### Add xarray dataset as an image service layer

In [ ]:
ds.leaflet.plot(m.map, lag=IS2widgets.lag,
    vmin=IS2widgets.vmin, vmax=IS2widgets.vmax,
    variable=IS2widgets.variable.value,
    cmap=IS2widgets.colormap,
    opacity=0.75,
    enable_popups=False)
IS2widgets.timelag.observe(ds.leaflet.set_lag)

### Remove the image service layer from the map

In [ ]:
ds.leaflet.reset()

### Extract time series from selected geometries
- Point: time series for a geolocation
- Transect: time series for a line segment
- Regional average: time series for a polygon or bounding box

In [ ]:
for feature in m.geometries['features']:
    ds.timeseries.plot(feature,
        variable=IS2widgets.variable.value,
    )